# Merge Datasets

In [41]:
import pandas as pd
import numpy as np
import re

### Load contracts

In [4]:
#### load data ####
df_con = pd.read_csv('../data/contracts.csv')

#### data cleaning ####
# remove diactrics
df_con['fullName'] = df_con['fullName'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

# convert cols to numeric where possible
cols = ['signAge', 'contractLength']
df_con[cols] = df_con[cols].apply(pd.to_numeric, errors='coerce')

# convert dollar to float
df_con[['totalValue', 'aav']] = df_con[['totalValue', 'aav']].replace('[\$,]', '', regex=True).astype(float)

# drop unnecessary cols
df_con.drop(['position', 'lastName', 'signBonus'], axis=1, inplace=True)

# write the season they signed in
df_con['term'] = df_con['term'].str.split().str[0]
df_con[['termStart', 'termEnd']] = df_con['term'].str.split('-', expand=True)
df_con['signSeason'] = (df_con['termStart'].astype(int)-1).astype(str)
df_con['termStart'] = df_con['termStart'].str[2:]
df_con['signSeason'] = df_con[['signSeason', 'termStart']].apply(lambda row: '-'.join(row.values.astype(str)), axis=1)

# drop unnecessary columns
df_con.drop(['termStart', 'termEnd'], axis=1, inplace=True)

In [5]:
df_con

,fullName,term,signAge,contractLength,totalValue,aav,isEntryLevel,signSeason
0,Carmelo Anthony,2014-2018,30.0,5,124064681.0,24812936.0,0,2013-14
1,Chris Bosh,2014-2018,29.0,5,118705300.0,23741060.0,0,2013-14
2,Paul George,2014-2018,23.0,5,91572660.0,18314532.0,0,2013-14
3,John Wall,2014-2018,22.0,5,84789500.0,16957900.0,0,2013-14
4,Eric Bledsoe,2014-2018,24.0,5,70000000.0,14000000.0,0,2013-14
...,...,...,...,...,...,...,...,...
2968,Jaime Echenique,2021-2021,24.0,1,925258.0,925258.0,0,2020-21
2969,Feron Hunt,2021-2021,22.0,1,925258.0,925258.0,0,2020-21
2970,Carlik Jones,2021-2021,23.0,1,925258.0,925258.0,0,2020-21
2971,Aric Holman,2021-2021,24.0,1,925258.0,925258.0,0,2020-21


### Load player stats

In [6]:
#### load data ####
df_stats = pd.read_csv('../data/playerStatsWithLags.csv')

#### data cleaning ####
# remove diactrics
df_stats['Name'] = df_stats['Name'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

# rename cols to match df_con
df_stats.rename(columns={'Name':'fullName', 'Season':'signSeason'}, inplace=True)

# drop unnecessary columns
df_stats.drop(['Age', 'Lg', 'Pos'], axis=1, inplace=True)

df_stats.head()

,fullName,signSeason,Tm,G,GS,MP,FG,FGA,FG%,3P,...,TOV%_prev2,USG%_prev2,OWS_prev2,DWS_prev2,WS_prev2,WS/48_prev2,OBPM_prev2,DBPM_prev2,BPM_prev2,VORP_prev2
0,Jeff Adrien,2010-11,GSW,23,0,8.5,1.0,2.3,0.426,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0
1,Jeff Adrien,2011-12,HOU,8,0,7.9,0.9,2.0,0.438,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0
2,Jeff Adrien,2012-13,CHA,52,5,13.7,1.4,3.2,0.429,0.0,...,12.6,16.1,0.1,0.1,0.2,0.051,-3.3,-2.0,-5.3,-0.2
3,Jeff Adrien,2013-14,TOT,53,12,18.1,2.7,5.2,0.520,0.0,...,8.6,16.8,0.1,0.1,0.1,0.106,-4.0,-0.2,-4.3,0.0
4,Jeff Adrien,2014-15,MIN,17,0,12.6,1.1,2.6,0.432,0.0,...,13.1,15.6,0.5,0.4,1.0,0.064,-2.1,-0.9,-3.0,-0.2


### Load player bios

In [82]:
df_bio = pd.read_csv('../data/playerBio.csv')
df_bio.rename({'name': 'fullName'}, axis=1, inplace=True)

# convert height/weight to ints
df_bio['height'] = pd.to_numeric(df_bio['height'].str.split('-', expand=True)[0])*12 + \
                    pd.to_numeric(df_bio['height'].str.split('-', expand=True)[1])
df_bio['weight'] = df_bio['weight'].str.extract('(\d+)')

# remove superscript ordinal from draft data
df_bio['draftRound']   = df_bio['draftRound'].str.extract('(\d+)')
df_bio['draftOverall'] = df_bio['draftOverall'].str.extract('(\d+)')

# correct draft data for undrafted players
df_bio['draftRound']   = np.where(df_bio['undrafted']==True, 0, df_bio['draftRound'])
df_bio['draftOverall'] = np.where(df_bio['undrafted']==True, 0, df_bio['draftOverall'])
df_bio['draftYear']    = np.where(df_bio['undrafted']==True, 0, df_bio['draftYear'])

# convert weight, draft data to numeric
cols = ['weight', 'draftRound', 'draftOverall', 'draftYear']
df_bio[cols] = df_bio[cols].apply(pd.to_numeric, errors='coerce')

# convert birthDate to datatime object
df_bio['birthDate'] = pd.to_datetime(df_bio['birthDate'])

df_bio

,fullName,height,weight,birthDate,draftRound,draftOverall,draftYear,undrafted
0,Jeff Adrien,79,245,1986-02-10,0,0,0,True
1,Arron Afflalo,77,210,1985-10-15,1,27,2007,False
2,Blake Ahearn,74,190,1984-05-27,0,0,0,True
3,Solomon Alabi,85,251,1988-03-21,2,50,2010,False
4,Cole Aldrich,83,250,1988-10-31,1,11,2010,False
...,...,...,...,...,...,...,...,...
1332,Patrick Williams,79,215,2001-08-26,1,4,2020,False
1333,Dylan Windler,78,196,1996-09-22,1,26,2019,False
1334,Cassius Winston,73,185,1998-02-28,2,53,2020,False
1335,James Wiseman,84,240,2001-03-31,1,2,2020,False


In [83]:
df_bio[df_bio['fullName']=='LeBron James']

,fullName,height,weight,birthDate,draftRound,draftOverall,draftYear,undrafted
233,LeBron James,81,250,1984-12-30,1,1,2003,False


### merge nonrookie data

In [86]:
df = df_con.merge(df_stats, on=['fullName', 'signSeason'], how='inner')
df = df.merge(df_bio, on='fullName', how='inner')

In [91]:
# df.info(verbose=True)

In [92]:
df.to_csv('../data/full_nonrookie.csv', index=False)

### merge rookie data

In [95]:
df_rookie = df_con[df_con['isEntryLevel']==1].merge(df_bio, on='fullName', how='inner')

In [101]:
# df_rookie[['fullName','aav', 'draftOverall', 'draftYear']].sort_values(['aav', 'draftOverall'], ascending=False).head(40)

In [102]:
df_rookie.to_csv('../data/full_rookie.csv', index=False)

which items are not getting merged

In [8]:
merged = set(df['fullName'])

In [9]:
names = [name for name in df_con['fullName'] if name not in merged]
# for name in df_con['fullName'].tolist():
#     if name not in merged:
#         print(name)
df_names = pd.DataFrame(names, columns=['fullName'])
df_names.merge(df_con, on='fullName')

,fullName,term,signAge,contractLength,totalValue,aav,isEntryLevel,signSeason
0,Stan Van Gundy,2014-2018,54.0,5,35000000.0,7000000.0,0,2013-14
1,Derek Fisher (Coach),2014-2018,NaN,5,25000000.0,5000000.0,0,2013-14
2,David Blatt,2014-2017,NaN,4,20000000.0,5000000.0,0,2013-14
3,Byron Scott,2014-2017,0.0,4,17000000.0,4250000.0,0,2013-14
4,Patrick Mills,2014-2016,25.0,3,13000000.0,4333333.0,0,2013-14
...,...,...,...,...,...,...,...,...
1664,Malcolm Hill,2021-2021,25.0,1,925258.0,925258.0,0,2020-21
1665,James Banks III,2021-2021,23.0,1,925258.0,925258.0,0,2020-21
1666,Jaime Echenique,2021-2021,24.0,1,925258.0,925258.0,0,2020-21
1667,Feron Hunt,2021-2021,22.0,1,925258.0,925258.0,0,2020-21


In [10]:
df_con[df_con['fullName']=='Kristaps Porzingis']

,fullName,term,signAge,contractLength,totalValue,aav,isEntryLevel,signSeason
428,Kristaps Porzingis,2015-2018,19.0,4,18650094.0,4662524.0,1,2014-15
1924,Kristaps Porzingis,2019-2023,23.0,5,158253000.0,31650600.0,0,2018-19


In [11]:
df_stats[df_stats['fullName']=='Kristaps Porzingis']

,fullName,signSeason,Tm,G,GS,MP,FG,FGA,FG%,3P,...,TOV%_prev2,USG%_prev2,OWS_prev2,DWS_prev2,WS_prev2,WS/48_prev2,OBPM_prev2,DBPM_prev2,BPM_prev2,VORP_prev2
6314,Kristaps Porzingis,2015-16,NYK,72,72,28.4,5.2,12.3,0.421,1.1,...,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0
6315,Kristaps Porzingis,2016-17,NYK,66,65,32.8,6.7,14.9,0.450,1.7,...,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0
6316,Kristaps Porzingis,2017-18,NYK,48,48,32.4,8.1,18.5,0.439,1.9,...,10.9,24.6,1.5,2.9,4.4,0.102,0.6,0.3,0.9,1.5
6317,Kristaps Porzingis,2019-20,DAL,57,57,31.8,7.0,16.4,0.427,2.5,...,9.7,24.3,2.4,2.1,4.5,0.100,0.7,-0.3,0.3,1.3
6318,Kristaps Porzingis,2020-21,DAL,43,43,30.9,7.6,15.9,0.476,2.3,...,8.4,31.0,1.4,1.8,3.2,0.099,1.9,-0.1,1.8,1.5
6319,Kristaps Porzingis,2021-22,TOT,51,51,29.0,6.9,15.0,0.459,1.5,...,8.0,27.0,2.6,2.6,5.2,0.138,1.8,0.1,1.9,1.8


In [88]:
df[df['fullName']=='John Wall']

,fullName,term,signAge,contractLength,totalValue,aav,isEntryLevel,signSeason,Tm,G,...,DBPM_prev2,BPM_prev2,VORP_prev2,height,weight,birthDate,draftRound,draftOverall,draftYear,undrafted
8,John Wall,2014-2018,22.0,5,84789500.0,16957900.0,0,2013-14,WAS,82,...,-0.6,0.7,1.6,75,210,1990-09-06,1,1,2010,False
9,John Wall,2019-2022,26.0,4,171131520.0,42782880.0,0,2018-19,WAS,32,...,0.1,4.7,4.8,75,210,1990-09-06,1,1,2010,False
